# Named Entities annotation using weak supervision

## Instalando bibliotecas

In [ ]:
!pip install --quiet -U spacy skweak

In [ ]:
!python -m spacy download 'pt_core_news_lg'

In [ ]:
import pandas as pd
import os
import spacy
import re
import skweak
from skweak import heuristics, gazetteers, aggregation, utils, base

## Carregando Dados

In [ ]:
folder = './'
folder_processados = f"{folder}/dados_processados"

if not os.path.exists(folder_processados):
    os.mkdir(folder_processados)

In [ ]:
df = pd.read_csv(folder+'/dataset.csv', sep=';')
df.head()

In [ ]:
dataset = [ s.replace('\n','').strip() for s in df['sentenca'].values ]
print(len(dataset))
dataset[:20]

## Skweak

* https://aclanthology.org/2021.acl-demo.40.pdf

* https://github.com/NorskRegnesentral/skweak

* https://github.com/NorskRegnesentral/skweak/wiki/Step-1:-Labelling-functions

* https://analyticsindiamag.com/meet-skweak-a-python-toolkit-for-applying-weak-supervision-to-nlp-tasks/

* https://colab.research.google.com/drive/1X90PP-sGbD5_TfXxigWI5_IQqWuuXrTE?usp=sharing#scrollTo=tZkFWMLytO9i

In [ ]:
spacy_model = "pt_core_news_lg"
nlp = spacy.load(spacy_model)

In [ ]:
# carregando rótulos e expressões a serem rotuladas
rotulos = gazetteers.extract_json_data(folder+"/gazetteers.json", spacy_model=spacy_model)

## Funções

### Money

In [ ]:
# Identifica um valor monetário
# Acrescentada a unidade: milhão/bilhão... Ver: https://github.com/NorskRegnesentral/skweak/wiki/Step-1:-Labelling-functions#heuristicsspaneditorannotator
def money_detector(doc):

  for tok in doc[1:]:
    if (tok.text in ["R$","US$"]) and tok.nbor(+1).like_num:
        yield tok.i, tok.i+2, 'MONEY'

def transform_money_span(span):
    last_token = span[-1]
    if last_token .n_rights and last_token.nbor(1).text in {"mil", "milhão", "milhões", "bilhão", "bilhões", "trilhão", "trilhões"}:
        return span.doc[span.start:span.end+1]

    return span



In [ ]:
# testes

#money_detector  = heuristics.FunctionAnnotator('money_number', money_detector)
#money_detector2 = heuristics.SpanEditorAnnotator("money", "money_number", transform_money_span)

money_detector  = heuristics.FunctionAnnotator('money_number', money_detector)
money = heuristics.SpanEditorAnnotator("money", "money_number", transform_money_span)

#p = "O lucro foi de R$ 13,8 bilhões ou quase US$3 bilhões "
p = "Paraná Banco - Teleconferência primeiro trimestre de 2009 – 08/05/2009 Operadora: Bom dia."
print(p)

doc = nlp(p)
doc = money_detector(doc)

for span in doc.spans['money_number']:
    print(f"{span.text} - {span.label_}")


utils.display_entities(doc, 'money_number')

doc = money(doc)

for span in doc.spans['money']:
    print(f"{span.text} - {span.label_}")


utils.display_entities(doc, 'money')

print(doc.to_json())

### Percentual

In [ ]:
def percent_detector(doc):

  for tok in doc[1:]:
    if tok.i < len(doc)-1 and tok.like_num and (tok.nbor(1).text in ["%", "bp","pp"]):
        yield tok.i, tok.i+2, 'PERCENTUAL'


In [ ]:
lf1 = heuristics.FunctionAnnotator('percent', percent_detector)

p = "O lucro foi de 13,8% maior ou quase 0,3 bp. equivalentes a 3 pp."
print(p)

doc = nlp(p)
doc = lf1(doc)

utils.display_entities(doc, 'percent')

### Regex

In [ ]:
p = "Obrigado por participarem da nossa 2T22 teleconferência 3T dos resultados do 2T2022"

print(p)
doc = nlp(p)

quarter = heuristics.TokenConstraintAnnotator('quarter', lambda tok: re.match('\dT(\d{2}|\d{4})', tok.text), 'QUARTER')

doc = quarter(doc)
#print(doc.to_json())
for s in doc.spans['quarter']:
    print(s.text, s.label_)


## Completo

In [ ]:
# salvando os documentos processados pelo Spacy
docs = [ nlp(s) for s in dataset ]
skweak.utils.docbin_writer(docs, f"{folder_processados}/file.spacy")

In [ ]:
# carregando os documentos processados pelo Spacy
docs = list(skweak.utils.docbin_reader(f"{folder_processados}/file.spacy", spacy_model_name=spacy_model))

In [ ]:
# Identifica um valor monetário
# Acrescentada a unidade: milhão/bilhão... Ver: https://github.com/NorskRegnesentral/skweak/wiki/Step-1:-Labelling-functions#heuristicsspaneditorannotator
def money_detector(doc):

  for tok in doc[1:]:
    if (tok.text in ["R$","US$"]) and tok.nbor(+1).like_num:
        yield tok.i, tok.i+2, 'MONEY'

def transform_money_span(span):
    last_token = span[-1]
    if last_token .n_rights and last_token.nbor(1).text in {"mil", "milhão", "milhões", "bilhão", "bilhões", "trilhão", "trilhões"}:
        return span.doc[span.start:span.end+1]

    return span




money_detector  = heuristics.FunctionAnnotator('money_number', money_detector)
money = heuristics.SpanEditorAnnotator("money", "money_number", transform_money_span)


percent = heuristics.FunctionAnnotator('percent', percent_detector)

quarter = heuristics.TokenConstraintAnnotator('quarter', lambda tok: re.match('\dT(\d{2}|\d{4})', tok.text), 'QUARTER')

semester = heuristics.TokenConstraintAnnotator('semester', lambda tok: re.match('\dS(\d{2}|\d{4})', tok.text), 'SEMESTER')

year = heuristics.TokenConstraintAnnotator('year', lambda tok: re.match('\d{4}$', tok.text), 'YEAR')

gaz = gazetteers.GazetteerAnnotator('rotulos', rotulos, case_sensitive=False)


combined = base.CombinedAnnotator()
combined.add_annotator(money_detector)
combined.add_annotator(money)
combined.add_annotator(percent)
combined.add_annotator(quarter)
combined.add_annotator(semester)
combined.add_annotator(year)
combined.add_annotator(gaz)

docs_anot = list(combined.pipe(docs))


In [ ]:
for d in docs_anot[:20]:
    print()
    #print(d)
    #print(d.spans)
    utils.display_entities(d, layer='*')

In [ ]:
# reunindo os labels das heuristicas aos labels do gazetteers
labels = ['MONEY', 'PERCENTUAL', 'QUARTER', 'SEMESTER', 'YEAR'] + list(rotulos.keys())
labels = list(set(labels)) # retirar os labels duplicados
print(len(labels))
print(labels)

In [ ]:
# retorna uma lista dos rótulos no formato BIO
label_names = {0: 'O'}
i = 1
for l in labels:
    label_names[i] = f"B-{l}"
    label_names[i+1] = f"I-{l}"
    i += 2


In [ ]:
# agregando rotulos
hmm  = aggregation.HMM('hmm', labels)
docs_agg = hmm.fit_and_aggregate(docs_anot)

for i, doc in enumerate(docs_agg[:20]):
  print(f"{i}")
  utils.display_entities(doc, 'hmm')

In [ ]:
for doc in docs_agg[:20]:
    print(doc.text)
    print(doc.spans["hmm"])

In [ ]:
import json

ls = [ d.to_json() for d in docs_agg ]
with open(folder_processados+"/dataset_annot.json", 'w', encoding='utf8') as json_file:
    json.dump(ls, json_file, indent=3, ensure_ascii=False)

## Analisando dados puros e anotados

* Quantidade de sentenças
    * Quantidade de Sentenças anotadas
    * Quantidade de Sentenças não anotadas
* Quantidade de anotações por Label
* Características das sentenças não anotadas


In [ ]:
# separando sentenças anotadas das não anotadas
# as sentenças anotadas serão repetidas para cada label encontrado nela
sents       = []
sents_annot = []
for d in docs_agg:

    if len(list(d.spans["hmm"])) > 0:
        for span in d.spans["hmm"]:
            row = {'text': d.text, 'token': span.text ,'label': span.label_}
            sents_annot.append(row)
    else:
        row = {'text': d.text}
        sents.append(row)

df_sents       = pd.DataFrame.from_dict(sents)
df_sents_annot = pd.DataFrame.from_dict(sents_annot)

In [ ]:
df_sents_annot

In [ ]:
print(f"Quantidade de sentenças: {len(docs)}")
print(f"Quantidade de sentenças não anotadas: {len(df_sents)}")

qtd_sent_anot = len(df_sents_annot['text'].unique())
print(f"Quantidade de sentenças anotadas: {qtd_sent_anot}")

qtd_anotacoes = len(df_sents_annot['label'])
print(f"Quantidade de anotações: {qtd_anotacoes}")

print(f"Média de anotações por sentença: {(qtd_anotacoes/qtd_sent_anot):.2f}")

print(f"Quantidade de anotações por label: ")
df_sents_annot['label'].value_counts()

In [ ]:
df_sents.to_csv(folder_processados+'/sentencas-nao-anotadas.csv', sep=';')
df_sents_annot.to_csv(folder_processados+'/sentencas-anotadas.csv', sep=';')

## Salvando dados no formato do Doccano

In [ ]:
# Adaptado de: https://stackoverflow.com/questions/57902256/how-to-export-document-with-entities-from-spacy-for-use-in-doccano
djson = list()
for doc in docs_agg:
    labels = list()
    for e in doc.spans['hmm']:
        labels.append([e.start_char, e.end_char, e.label_])
    djson.append({'text': doc.text, "label": labels})

djson

In [ ]:
import json

open(folder_processados+'/doccano.json', 'w', encoding='utf8').write("\n".join([json.dumps(e) for e in djson]))

## Carregando dados do Doccano

In [ ]:
import pandas as pd


In [ ]:
jsonObj = pd.read_json(path_or_buf=f"{folder_processados}/doccano-20231004.jsonl", lines=True)
jsonObj

In [ ]:
sents_anot = []
sents_anot_aux = []
sents_sem_anot = []
qtd_sent_anot = 0
qtd_sent_sem_anot = 0
for idx in range(0, len(jsonObj)):
#for idx in range(0,5):
    text = jsonObj.loc[idx, 'text']
    labels = jsonObj.loc[idx, 'label']
    if len(labels) > 0:
        for span in labels:
            token = text[span[0]:span[1]]
            label = span[2]
            row = {'text': text, 'token': token ,'label': label}
            sents_anot.append(row)
        qtd_sent_anot += 1
        sents_anot_aux.append(text)
    else:
        row = {'text': text, 'token': '' ,'label': ''}
        sents_sem_anot.append(row)
        qtd_sent_sem_anot += 1

df_sents_anot = pd.DataFrame.from_dict(sents_anot)
df_sents_anot

In [ ]:
df_sents_sem_anot = pd.DataFrame.from_dict(sents_sem_anot)
df_sents_sem_anot

In [ ]:
qtd_sent_anot, qtd_sent_sem_anot

In [ ]:
len(df_sents_anot), len(df_sents_sem_anot)

In [ ]:
sents_unique = df_sents_anot['text'].unique()

In [ ]:
len(sents_unique), len(sents_anot_aux)

In [ ]:
df_aux = pd.DataFrame()
df_aux['text'] = sents_anot_aux
#df_aux.sort_values(by='text')
pd.concat(g for _, g in df_aux.groupby("text") if len(g) > 1)

In [ ]:
print(f"Quantidade de sentenças original: {len(jsonObj)}")

print(f"Quantidade de sentenças não anotadas: {len(df_sents_sem_anot['text'])}")

qtd_sent_anot = len(df_sents_anot['text'].unique())
print(f"Quantidade de sentenças anotadas: {qtd_sent_anot}")

qtd_anotacoes = len(df_sents_anot['label'])
print(f"Quantidade de anotações: {qtd_anotacoes}")

print(f"Média de anotações por sentença: {(qtd_anotacoes/qtd_sent_anot):.2f}")

print(f"Quantidade de anotações por label: ")
df_sents_anot['label'].value_counts()

In [ ]:
df_sents_anot.to_csv(folder_processados+'/sentencas-anotadas-doccano.csv', sep=';', index=False)